In [ ]:
import tensorflow as tf
import keras
from keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.optimizers import Adam
import cv2
import os
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, roc_auc_score

In [ ]:
dataset_path = 'Datasets/improvised_images/'

In [ ]:
image_size = (32,32)
batch_size = 32

#Data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_batches = train_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    color_mode = 'grayscale',
    class_mode='sparse',
    subset='training'
)

validation_batches = train_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    color_mode = 'grayscale',
    class_mode='sparse',
    subset='validation'
)

test_batches = train_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    color_mode = 'grayscale',
    class_mode='sparse',
    shuffle=False,
    subset='validation'
)

In [ ]:
train_batches.class_indices

In [ ]:
imp_characters = ['क','ख','ग','घ','ङ','च','छ','ज','झ','ञ','ट','ठ','ड','ढ','ण','त','थ','द','ध','न','प','फ','ब',
                'भ','म','य','र','ल','व','श','ष','स','ह','क्ष','त्र','ज्ञ','अ','इ','उ','ऊ','ऋ','ए','क्र','ट्र','त्त','द्य',
                'प्र','श्र','रु','८']

In [ ]:
def plotImages(image_arr):
    fig, axes = plt.subplots(1,5)
    axes = axes.flatten()
    for img, ax in zip(image_arr, axes):
        ax.imshow(img[:,:,0])
        ax.axis('off')
    plt.tight_layout()
    plt.show()

imgs, labels = train_batches[0]
plotImages(imgs)
for i in range(5):
  print(imp_characters[int(labels[:5][i])],end='\t')

In [ ]:
imgs.shape

In [ ]:
labels.shape

In [ ]:
model1 = Sequential()

model1.add(Conv2D(128, (3, 3), strides = 1, activation = 'relu', input_shape = (32, 32, 1)))
model1.add(MaxPooling2D((2, 2), strides = (2, 2), padding = 'same'))
model1.add(Dropout(0.25))

model1.add(Conv2D(128, (3, 3), strides = 1, activation = 'relu'))
model1.add(MaxPooling2D((2, 2), strides = (2, 2), padding = 'same'))
model1.add(Dropout(0.25))

model1.add(Conv2D(128, (3, 3), strides = 1, activation = 'relu'))
model1.add(MaxPooling2D((2, 2), strides = (2, 2), padding = 'same'))
model1.add(Dropout(0.25))

model1.add(Flatten())

model1.add(Dense(256, activation = 'relu', kernel_initializer = 'he_uniform'))
model1.add(Dropout(0.2))

model1.add(Dense(128, activation = 'relu', kernel_initializer = 'he_uniform'))
model1.add(Dropout(0.2))

model1.add(Dense(50, activation = 'softmax'))

In [ ]:
model1.summary()

In [ ]:
model1.compile(optimizer = Adam(learning_rate = 1e-3, decay = 1e-5), 
               loss = 'sparse_categorical_crossentropy', 
               metrics = ['accuracy'])

In [ ]:
if not os.path.isdir('Models/imp_Model'):
    os.mkdir('Models/imp_Model')

In [ ]:
callbacks1 = [ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1,
                              patience = 7, min_lr = 1e-5),
             EarlyStopping(monitor = 'val_loss', patience = 9, # Patience should be larger than the one in ReduceLROnPlateau
                          min_delta = 1e-5),
             CSVLogger(os.path.join('Models/imp_Model','m_training_1.csv'), separator =',', append = True),
             ModelCheckpoint(os.path.join('Models/imp_Model', 'backup_last_model.hdf5')),
             ModelCheckpoint(os.path.join('Models/imp_Model', 'best_val_acc.hdf5'), monitor = 'val_accuracy', mode = 'max', save_best_only = True),
             ModelCheckpoint(os.path.join('Models/imp_Model', 'best_val_loss.hdf5'), monitor = 'val_loss', mode = 'min', save_best_only = True)]

In [ ]:
model1.fit(train_batches, epochs = 30, validation_data = validation_batches, callbacks = callbacks1)

In [ ]:
model1 = load_model(os.path.join('Models/imp_Model', 'best_val_loss.hdf5'))

In [ ]:
i=0
pred1=[]
num1=model1.predict(test_batches) 
l=len(num1)
#print(num1)
for i in range(l):
    val=np.argmax(num1[i])
    i+=1
    pred1.append(val)

In [ ]:
m_cm1=confusion_matrix(test_batches.classes,pred1)
print(m_cm1)